In [19]:
from astropy.io import fits
from astropy.table import Table
from astropy.io import ascii
import numpy as np

We are going to read in a fits table that contains info on galaxies in the region of the Virgo cluster.

In [30]:
vfile = 'tables/nsa.virgo.fits'
vdat = fits.getdata(vfile)

This table has a lot of columns.  The columns are defined here: http://nsatlas.org/data

To view the column names:

In [31]:
vdat.columns

ColDefs(
    name = 'IAUNAME'; format = '19A'
    name = 'SUBDIR'; format = '27A'
    name = 'RA'; format = 'D'
    name = 'DEC'; format = 'D'
    name = 'ISDSS'; format = 'J'
    name = 'INED'; format = 'J'
    name = 'ISIXDF'; format = 'J'
    name = 'IALFALFA'; format = 'J'
    name = 'IZCAT'; format = 'J'
    name = 'ITWODF'; format = 'J'
    name = 'MAG'; format = 'E'
    name = 'Z'; format = 'E'
    name = 'ZSRC'; format = '7A'
    name = 'SIZE'; format = 'E'
    name = 'RUN'; format = 'I'
    name = 'CAMCOL'; format = 'B'
    name = 'FIELD'; format = 'I'
    name = 'RERUN'; format = '3A'
    name = 'XPOS'; format = 'E'
    name = 'YPOS'; format = 'E'
    name = 'ZLG'; format = 'E'
    name = 'ZDIST'; format = 'E'
    name = 'ZDIST_ERR'; format = 'E'
    name = 'NSAID'; format = 'J'
    name = 'NMGY'; format = '7E'
    name = 'NMGY_IVAR'; format = '7E'
    name = 'OK'; format = 'I'
    name = 'RNMGY'; format = '7E'
    name = 'ABSMAG'; format = '7E'
    name = 'AMIVAR'; format = 

For a void-finding algorithm, we want to print out the RA, DEC and recession velocity.  The catalog I read in has redshift, so we need to calculate recession velocity:

$$ v_r = z \times c $$

In [32]:
from astropy import constants as const

In [33]:
const.c.to('km/s')

<Quantity 299792.458 km / s>

We can now calculate recession velocity from the redshift (column ZDIST)

In [34]:
vr = vdat.ZDIST*const.c.to('km/s')

We need to write a table with 7 columns.  The columns are

* 1 - index
* 2 - not used, but we will use the NSA ID
* 3 - not used, but we need to use something as a place holder
* 4 - RA
* 5 - DEC
* 6 - recession velocity (km/s)
* 7 - magnitude
* 8 - not used, but we need to use something as a place holder

We need to define the index and the magnitude first:

In [35]:
index = np.arange(len(vdat.ZDIST))
# use r-band magnitude from NSA catalog
mag = 22.5 - 2.5*np.log10(vdat.NMGY[:,4])


/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: RuntimeWarning: invalid value encountered in log10
  app.launch_new_instance()


We are going to use astropy.table to create an ascii table.

In [39]:
cols = [index,vdat.NSAID,vdat.NSAID,vdat.RA,vdat.DEC, vr, mag, mag]
colnames = ['index', 'nsaid', 'nsaid1','RA','DEC','vr','mag','mag1']
newtable = Table(cols, names=colnames)
newtable.write('tables/virgovoids.dat',overwrite=True,format='ascii.tab')
